## 기본 합성곱 신경망 구현


In [1]:
import tensorflow as tf
import numpy as np

## 하이퍼 파라미터

In [2]:
EPOCHS = 10

## 모델 정의 (VGG-16 형태)
#### tf.keras.layers.Conv2D(16,(3,3),padding='same',activation='relu') : Convoultion 
0. 입력 채널 갯수는 API가 저절로 판별
1. 16 : 출력 채널 갯수
2. (3,3) : Convolution kernel 크기 설정
3. padding = 'same' -> 'valid' 시 zero padding 안해줌 (크기가 줄어든다)

#### tf.keras.layers.MaxPool2D((2,2)) : Pooling
1. 2x2 maxpool 방식 사용

In [3]:
class ConvNet(tf.keras.Model):
    def __init__(self):
        super(ConvNet, self).__init__()
        conv2d = tf.keras.layers.Conv2D
        maxpool = tf.keras.layers.MaxPool2D
        self.sequence = list()
        self.sequence.append(conv2d(16, (3, 3), padding='same', activation='relu')) # 28x28x16
        self.sequence.append(conv2d(16, (3, 3), padding='same', activation='relu')) # 28x28x16
        self.sequence.append(maxpool((2,2))) # 14x14x16
        self.sequence.append(conv2d(32, (3, 3), padding='same', activation='relu')) # 14x14x32
        self.sequence.append(conv2d(32, (3, 3), padding='same', activation='relu')) # 14x14x32
        self.sequence.append(maxpool((2,2))) # 7x7x32
        self.sequence.append(conv2d(64, (3, 3), padding='same', activation='relu')) # 7x7x64
        self.sequence.append(conv2d(64, (3, 3), padding='same', activation='relu')) # 7x7x64
        self.sequence.append(tf.keras.layers.Flatten()) # 7x7x64 = 1568 (벡터화)
        self.sequence.append(tf.keras.layers.Dense(128, activation='relu'))
        self.sequence.append(tf.keras.layers.Dense(10, activation='softmax'))

    def call(self, x, training=False, mask=None):
        for layer in self.sequence:
            x = layer(x)
        return x

## 학습, 테스트 루프 정의

In [4]:
# Implement training loop
@tf.function
def train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)

    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)

# Implement algorithm test
@tf.function
def test_step(model, images, labels, loss_object, test_loss, test_accuracy):
    predictions = model(images)

    t_loss = loss_object(labels, predictions)
    test_loss(t_loss)
    test_accuracy(labels, predictions)

## 데이터셋 준비
##### numpy[...] 의미 : numpy[:,:,:]
1. [..., tf.newaxis] -> 한 차원 증가시키기
2. 한차원 증가 시킨 이유 : Convolution Layer 들어갈 시 채널 갯수를 명시하기 위함

##### tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
1. Dataset.from_tensor_slices : 클레스 메소드로서 np,tf 자료형 데이터 셋 만들 수 있다.
2. shuffle : Overfitting을 피하기 위하여 DataSet섞는다. 
3. batch : 배 트레이닝 -> 배치 데이터 보고 전체 데이터에 적용

In [5]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# x_train : (NUM_SAMPLE, 28, 28) -> (NUM_SAMPLE, 28, 28, 1)
x_train = x_train[..., tf.newaxis].astype(np.float32)
x_test = x_test[..., tf.newaxis].astype(np.float32)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

## 학습 환경 정의
### 모델 생성, 손실함수, 최적화 알고리즘, 평가지표 정의

In [6]:
# Create model
model = ConvNet()

# Define loss and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Define performance metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

## 학습 루프 동작

In [7]:
for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)

    for test_images, test_labels in test_ds:
        test_step(model, test_images, test_labels, loss_object, test_loss, test_accuracy)

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch + 1,
                          train_loss.result(),
                          train_accuracy.result() * 100,
                          test_loss.result(),
                          test_accuracy.result() * 100))
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

Epoch 1, Loss: 0.12333561480045319, Accuracy: 96.15666961669922, Test Loss: 0.04355228319764137, Test Accuracy: 98.68999481201172
Epoch 2, Loss: 0.0427783839404583, Accuracy: 98.72999572753906, Test Loss: 0.027533281594514847, Test Accuracy: 99.08999633789062
Epoch 3, Loss: 0.03273820877075195, Accuracy: 98.99666595458984, Test Loss: 0.029636705294251442, Test Accuracy: 99.08999633789062
Epoch 4, Loss: 0.023212188854813576, Accuracy: 99.25833129882812, Test Loss: 0.04126732051372528, Test Accuracy: 98.97000122070312
Epoch 5, Loss: 0.020038031041622162, Accuracy: 99.39500427246094, Test Loss: 0.026049906387925148, Test Accuracy: 99.22999572753906
Epoch 6, Loss: 0.017084360122680664, Accuracy: 99.45166778564453, Test Loss: 0.03556510806083679, Test Accuracy: 99.02999877929688
Epoch 7, Loss: 0.014365429989993572, Accuracy: 99.56666564941406, Test Loss: 0.040972765535116196, Test Accuracy: 98.94000244140625
Epoch 8, Loss: 0.01382648665457964, Accuracy: 99.56999969482422, Test Loss: 0.03112